In [4]:
import pandas as pd
import numpy as np
import pickle
pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr'

In [2]:
# store data files in  variables. all named DA_$file_name
data_files = ["aisles",
                "departments",
                "order_products__prior",
                "orders",
                "products",
                "sample_submission"
                ]
for file_name in data_files :
    exec( "DA_{0} = pd.read_csv('./data/{0}.csv')".format(file_name) )

# keep the columns i'm interested in only to reduce size.
#DA_RESIZED_order_products__prior = DA_order_products__prior.drop(['add_to_cart_order','reordered'],axis=1)
#DA_RESIZED_orders = DA_orders.drop(['user_id','eval_set','order_number','days_since_prior_order'],axis=1)
# and take sample for testing.    
#DA_SAMPLE_orders = DA_orders.loc[1:1000]
#DA_SAMPLE_order_products_prior = DA_RESIZED_order_products__prior[1:10000]

In [3]:
# row 0 should be empty.
# 1 to 49688 product rows
# 0 to 167 scores columns by hour, computed by : day * 24 + hour
#	where "day" in range 0 to 6 , and hour in range 0 to 23.
products_freq = np.zeros(shape=[49689,168])

In [ ]:
# will take 37 hours
t = np.datetime64('now')

def set_freq(product, col_id):
     products_freq[product.product_id][int(col_id)] += 1;

def get_products(order):
    products= DA_RESIZED_order_products__prior[DA_RESIZED_order_products__prior.order_id == order.order_id]
    products.apply(set_freq, col_id = (order.order_dow * 24) + order.order_hour_of_day, axis=1)
    
DA_SAMPLE_orders.apply(get_products, axis=1)
    
t = np.datetime64('now') - t
print t   

In [97]:
# reading/writing to/from pickle files.
pickle_file = './data/product_freq5.pkl'
pickle_file_backup = './data/backup/product_freq5.pkl.backup'
with open(pickle_file, 'wb') as f:
    pickle.dump(products_freq, f)
with open(pickle_file_backup, 'wb') as f:
    pickle.dump(products_freq, f)
#with open(filee, 'rb') as f:
#    tmp = pickle.load(f)

In [5]:
#products_S = DA_order_products__prior.query("product_id <= 1000")
#orders_S = DA_orders[DA_orders.order_id.isin(products_S.order_id)]

orders_S = DA_orders[DA_orders.user_id.isin(range(10001,50000))]
products_S = DA_order_products__prior[ DA_order_products__prior.order_id.isin(orders_S.order_id) ]

orders_S.drop(['user_id','eval_set','order_number','days_since_prior_order'],axis=1,inplace=1)
products_S.drop(['add_to_cart_order','reordered'],axis=1,inplace=1)

In [98]:
products_freq_backup = products_freq.copy()

In [24]:
# will take 37 hours
t = np.datetime64('now')

def set_freq(product, col_id):
     products_freq[product.product_id][int(col_id)] += 1;

def get_products(order):
    products= products_S[products_S.order_id == order.order_id]
    products.apply(set_freq, col_id = (order.order_dow * 24) + order.order_hour_of_day, axis=1)
    
orders_S.apply(get_products, axis=1)
    
t = np.datetime64('now') - t
print t   

5975 seconds


In [25]:
range(10001,50000)[-1]

49999

In [96]:
tt = np.datetime64('now');
step = 1000
for i in range(200000,207000,step):
    t = np.datetime64('now');
    
    orders_S = DA_orders[DA_orders.user_id.isin(range(i,i+step))]
    if len(orders_S) == 0 :
        print "finished";
        break;
    products_S = DA_order_products__prior[ DA_order_products__prior.order_id.isin(orders_S.order_id) ]

    orders_S.drop(['user_id','eval_set','order_number','days_since_prior_order'],axis=1,inplace=1)
    products_S.drop(['add_to_cart_order','reordered'],axis=1,inplace=1)
    
    
    #***************************************************************
    def set_freq(product, col_id):
        products_freq[product.product_id][int(col_id)] += 1;

    def get_products(order):
        products= products_S[products_S.order_id == order.order_id]
        products.apply(set_freq, col_id = (order.order_dow * 24) + order.order_hour_of_day, axis=1)
    
    orders_S.apply(get_products, axis=1)
    #*************************************************************
    
    t = np.datetime64('now') - t
    t = int(str(t).split(' ')[0])
    print  "\n* completed from {0} to {1}, with #orders = {2},#products = {3},\
 time_in_sec = {4}/ in minutes = {5}.".format(i, i+step-1, len(orders_S), len(products_S), t, t/60.0)
    
tt = np.datetime64('now') - tt;
tt = int(str(tt).split(' ')[0])
print "\n\n* total time in seconds : {0} or in minutes: {1}".format(tt, tt/60.0)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



* completed from 200000 to 200999, with #orders = 16070,#products = 150906, time_in_sec = 22/ in minutes = 0.366666666667.

* completed from 201000 to 201999, with #orders = 16888,#products = 155192, time_in_sec = 23/ in minutes = 0.383333333333.

* completed from 202000 to 202999, with #orders = 16854,#products = 154202, time_in_sec = 22/ in minutes = 0.366666666667.

* completed from 203000 to 203999, with #orders = 16921,#products = 156931, time_in_sec = 22/ in minutes = 0.366666666667.

* completed from 204000 to 204999, with #orders = 16557,#products = 156148, time_in_sec = 22/ in minutes = 0.366666666667.

* completed from 205000 to 205999, with #orders = 16282,#products = 152874, time_in_sec = 21/ in minutes = 0.35.

* completed from 206000 to 206999, with #orders = 3312,#products = 31200, time_in_sec = 6/ in minutes = 0.1.


* total time in seconds : 138 or in minutes: 2.3


In [40]:
t = np.datetime64('now') - t

In [63]:
int(str(t).split(' ')[0])

47

In [89]:
140000+1000

141000

In [84]:
log_file = "./data/log"
logg = open(log_file, 'w')


In [104]:
len(products_freq_backup[0])

168

In [3]:
DA_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0
